In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [67]:
df

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6,gas,145000,automatic,SUV,NaN,Yes,2018-06-23,19
1,25500,0,ford f-150,good,6,gas,88705,automatic,pickup,white,Yes,2018-10-19,50
2,5500,2013,hyundai sonata,like new,4,gas,110000,automatic,sedan,red,No,2019-02-07,79
3,1500,2003,ford f-150,fair,8,gas,0,automatic,pickup,NaN,No,2019-03-22,9
4,14900,2017,chrysler 200,excellent,4,gas,80903,automatic,sedan,black,No,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013,nissan maxima,like new,6,gas,88136,automatic,sedan,black,No,2018-10-03,37
51521,2700,2002,honda civic,salvage,4,gas,181500,automatic,sedan,white,No,2018-11-14,22
51522,3950,2009,hyundai sonata,excellent,4,gas,128000,automatic,sedan,blue,No,2018-11-15,32
51523,7455,2013,toyota corolla,good,4,gas,139573,automatic,sedan,black,No,2018-07-02,71


In [68]:
df = pd.read_csv("csv/vehicles_us.csv")
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013.0,nissan maxima,like new,6.0,gas,88136.0,automatic,sedan,black,NaN,2018-10-03,37
51521,2700,2002.0,honda civic,salvage,4.0,gas,181500.0,automatic,sedan,white,NaN,2018-11-14,22
51522,3950,2009.0,hyundai sonata,excellent,4.0,gas,128000.0,automatic,sedan,blue,NaN,2018-11-15,32
51523,7455,2013.0,toyota corolla,good,4.0,gas,139573.0,automatic,sedan,black,NaN,2018-07-02,71


In [75]:
df[df['paint_color'].isna()]

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6,gas,145000,automatic,SUV,NaN,Yes,2018-06-23,19
3,1500,2003,ford f-150,fair,8,gas,0,automatic,pickup,NaN,No,2019-03-22,9
8,11500,2012,kia sorento,excellent,4,gas,104174,automatic,SUV,NaN,Yes,2018-07-16,19
12,18990,2012,ram 1500,excellent,8,gas,140742,automatic,pickup,NaN,Yes,2019-04-02,37
21,5250,2007,toyota rav4,good,6,gas,154000,automatic,SUV,NaN,No,2018-08-22,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51502,7995,2011,chevrolet equinox,like new,4,gas,111088,automatic,SUV,NaN,No,2019-04-01,175
51503,6595,2011,chevrolet malibu,excellent,4,gas,100780,automatic,sedan,NaN,No,2019-02-27,73
51506,28900,2017,ford f150 supercrew cab xlt,good,6,gas,49225,automatic,pickup,NaN,Yes,2019-04-11,25
51519,11750,2015,honda accord,excellent,4,gas,0,automatic,coupe,NaN,No,2018-11-24,89


In [ ]:
def cleaning(data):
    data['is_4wd'] = data['is_4wd'].replace({1: 'Yes', np.nan: 'No'})
    data['date_posted'] = pd.to_datetime(data['date_posted'])
    data['odometer'] = data['odometer'].fillna(0)
    data['odometer'] = data['odometer'].astype(int)
    data['cylinders'] = data['cylinders'].fillna(0)
    data['cylinders'] = data['cylinders'].astype(int)
    data['model_year'] = data['model_year'].fillna(0)
    data['model_year'] = data['model_year'].astype(int)
    df["cylinders"] = df.groupby("model")["cylinders"].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan))
    data['paint_color'] = data['paint_color'].fillna('random')
    return data


cleaning(df)
df

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011,bmw x5,good,6,gas,145000,automatic,SUV,random,Yes,2018-06-23,19
1,25500,0,ford f-150,good,6,gas,88705,automatic,pickup,white,Yes,2018-10-19,50
2,5500,2013,hyundai sonata,like new,4,gas,110000,automatic,sedan,red,No,2019-02-07,79
3,1500,2003,ford f-150,fair,8,gas,0,automatic,pickup,random,No,2019-03-22,9
4,14900,2017,chrysler 200,excellent,4,gas,80903,automatic,sedan,black,No,2019-04-02,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51520,9249,2013,nissan maxima,like new,6,gas,88136,automatic,sedan,black,No,2018-10-03,37
51521,2700,2002,honda civic,salvage,4,gas,181500,automatic,sedan,white,No,2018-11-14,22
51522,3950,2009,hyundai sonata,excellent,4,gas,128000,automatic,sedan,blue,No,2018-11-15,32
51523,7455,2013,toyota corolla,good,4,gas,139573,automatic,sedan,black,No,2018-07-02,71
